### Simple signalling game tutorial

This file gives a very practical introducation, but you could go the the files that are referenced to see how to implement everything. 


### The dataset
First we load the dataset and inspect how it looks. We made two datasets of shapes of different colors. One for classification and one for the game itself.
Let us first inspect the classification dataset. 

This dataset consists of pill images with corresponding classes.

In [7]:
from shapeDataset import ShapeDataset
from IPython.display import Image 

shape_dataset = ShapeDataset()

example, label = next(iter(shape_dataset))
print(label)
display(example)

6


Next we look at the game dataset, it is simply a dataset that consists of 1 sender image, n receiver images and the index of the target.

In [11]:
from shapeDataset import ShapeGameDataset
shape_game_dataset = ShapeGameDataset()

sender_img, receiver_imgs, target = next(iter(shape_game_dataset))

print('sender img')
display(sender_img)

print('receiver_imgs')
for img in receiver_imgs:
    display(img)
    
print('target', target)




sender img


receiver_imgs


target 2


To make the dataset trainable we need to convert the images to tensors and use a dataloader. Luckely pytorch has us covered

In [12]:
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

batch_size=32

transform = transforms.Compose([transforms.ToTensor()])
signalling_game_train = ShapeGameDataset(transform=transform, samples_per_epoch=10000)
train_dataloader = DataLoader(signalling_game_train, shuffle=True, batch_size=batch_size, )

### The game
Next we show how you can train a model that uses Gumbel Softmax and sends discrete messages

In [23]:
import torch
from models.models import HiddenStateModel, SenderModelFixedLength, ReceiverModuleFixedLength
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#We first initialize the hidden state models 
#We do this because we want to be able to pretrain the hidden state model with the classification task (Check the model for more details)

hidden_state_model_sender = HiddenStateModel(9, input_channels=3) #There are 9 classes 3 colors * 3 shapes
hidden_state_model_receiver = HiddenStateModel(9, input_channels=3)


# Next we initialze the fixed length models. 
msg_len=5
n_symbols=3

sender = SenderModelFixedLength(10, n_symbols=n_symbols, msg_len=msg_len,
                                        hidden_state_model=hidden_state_model_sender).to(device)
receiver = ReceiverModuleFixedLength(10, n_symbols=n_symbols, msg_len=msg_len,
                                     hidden_state_model=hidden_state_model_receiver).to(device)




In [25]:
# We can checkout the output of the sender and the receiver 


for sender_img, receiver_imgs, target in train_dataloader:
    sender_img = sender_img.to(device)
    receiver_imgs = [receiver_img.to(device) for receiver_img in receiver_imgs]
    
    msg = sender(sender_img)
    print(msg) 
    out, out_probs = receiver(receiver_imgs, msg)
    print(out_probs)
    break
    

tensor([[[0., 1., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [1., 0., 0.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 1., 0.]],

        [[0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 1., 0.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [0., 1., 0.]],

        [[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [1., 0., 0.]],

        [[0., 1., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 1., 0.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 

As you can see the msg are one hot encodings.
The receiver predicts what the corresponding img is. 

In [26]:
### Next we initialize the loss and the training objective.

loss_module = torch.nn.CrossEntropyLoss()
parameters = list(sender.parameters()) + list(receiver.parameters())

optimizer = torch.optim.Adam(
            parameters,
            lr=0.0001)

In [37]:
### The actual training loop
n_epochs = 50

# The actual training:
for i in range(n_epochs):
    batch_count = 0
    train_accuracy = 0
    for sender_img, receiver_imgs, target in train_dataloader:
        sender_img = sender_img.to(device)
        receiver_imgs = [receiver_img.to(device) for receiver_img in receiver_imgs]
        target = target.to(device)
        msg = sender(sender_img)

        out, out_probs = receiver(receiver_imgs, msg)
        loss = loss_module(out_probs, target)
        predicted_indices = torch.argmax(out_probs, dim=-1)
        
        receiver.zero_grad()
        sender.zero_grad()
        loss.backward()
        optimizer.step()
        correct = (predicted_indices == target).sum().item()
        train_accuracy += correct / len(target)
        batch_count += 1
        
    print(train_accuracy/batch_count)

0.541015625
0.5439453125
0.5478515625
0.5615234375
0.5791015625
0.5830078125
0.5986328125
0.59765625
0.626953125
0.6298828125
0.6357421875
0.6630859375
0.666015625
0.6865234375
0.69140625
0.705078125
0.6962890625
0.734375
0.744140625
0.7490234375
0.75390625
0.7568359375
0.771484375
0.7900390625
0.794921875
0.7939453125
0.796875
0.7998046875
0.810546875
0.8203125
0.81640625
0.8232421875
0.82421875
0.826171875
0.8212890625
0.830078125
0.833984375
0.8369140625
0.8310546875
0.833984375
0.8369140625
0.8466796875
0.8388671875
0.8408203125
0.849609375
0.849609375
0.8583984375
0.8525390625
0.8486328125
0.853515625


In [38]:
# We can no see how it changed, note that the model becomes a lot more certain about its predictions:
for sender_img, receiver_imgs, target in train_dataloader:
    sender_img = sender_img.to(device)
    receiver_imgs = [receiver_img.to(device) for receiver_img in receiver_imgs]
    
    msg = sender(sender_img)
    print(msg) 
    out, out_probs = receiver(receiver_imgs, msg)
    print(out_probs)
    break

tensor([[[1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.]],

        [[1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.]],

        [[1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.]],

        [[1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 

To see more details on how to do this with pytorch lighting see for example run_pl_model_fixed_size.py file and follow all the references to the files